# Statistical Analysis of Empiricist Experiment Data

In [10]:
import json
import statistics

Parsing the original analysis_results.jsonl to extract o1 high and o1 low results into separate files, assuming that these results are from evaluating with Sonnet 3.5 as the judge model.

In [20]:
official_results = '../analysis_results.jsonl'

# parse for o1 high results
o1_high_results = []
with open(official_results, 'r') as f:
    for line in f:
        data = json.loads(line)
        if data['model'] == 'o1_maxiter_30_N_v0.20.0-no-hint-run_1':
            o1_high_results.append(data)

# save to file for later
with open('official/o1_high_results.jsonl', 'w') as f:
    for result in o1_high_results:
        f.write(json.dumps(result) + '\n')

o1_low_results = []
with open(official_results, 'r') as f:
    for line in f:
        data = json.loads(line)
        if data['model'] == 'o1_low_maxiter_30_N_v0.20.0-no-hint-run_1':
            o1_low_results.append(data)

# save to file for later
with open('official/o1_low_results.jsonl', 'w') as f:
    for result in o1_low_results:
        f.write(json.dumps(result) + '\n')

len(o1_high_results), len(o1_low_results)

(200, 199)

Quick function to calculate mean and standard deviation of overthinking scores from a given file.

In [22]:
def calc_stats(filepath, sample_size=None):
    scores = []

    with open(filepath, 'r') as f:
        for line in f:
            data = json.loads(line)
            scores.append(float(data['overthinking_score']))

    if sample_size is not None:
        scores = scores[:sample_size]

    avg = statistics.mean(scores)
    std_dev = statistics.stdev(scores)

    return avg, std_dev, len(scores)

Next, I analyze both official and my_eval files for o1 high and o1 low results to verify the author's claim that overthinking is more prevalent in o1_low model than in o1_high model. I use a sample size of 50 due to limited data in my_eval files.

my_eval files were evaluated using Anthropic's Haiku 3.5 model instead of Sonnet 3.5. I assumed that the Haiku model would perform similarly to Sonnet as an LLM-as-a-judge. It was used to cut costs and speed up the evaluation process.

In [28]:
# Analyze files
files = [
    'official/o1_high_results.jsonl',
    'official/o1_low_results.jsonl',
    'my_eval/o1_high_overthinking.jsonl',
    'my_eval/o1_low_overthinking.jsonl'
]

for filepath in files:
    avg, std_dev, count = calc_stats(filepath, sample_size=50)
    print(f"{filepath}: {avg:.02f} ± {std_dev:.02f}")

official/o1_high_results.jsonl: 1.06 ± 0.89
official/o1_low_results.jsonl: 3.46 ± 3.57
my_eval/o1_high_overthinking.jsonl: 2.98 ± 2.51
my_eval/o1_low_overthinking.jsonl: 3.00 ± 2.16


The next steps are to evaluate overthinking scores from a different model. There were several ways I thought about to force "low reasoning effort" and "high reasoning effort". At first, I attempted to use Deepseek-R1-Distill-Qwen-14B, since the model's trajectories are already provided in the repository. I considered these trajectories as "high reasoning effort" as I assumed that it had an unlimited token budget. My thinking was that I could run Deepseek-R1-Distill-Qwen-14B with max_token set for my "low reasoning effort" trajectories.

Since I was going to use the OpenHands framework to evaluate the model using SWE-Bench, I had to set this up first. However this is where I hit my first major roadblock. Since OpenHands relies on docker, I couldn't run OpenHands on KOA (HPC server), meaning I needed to host the model remotely and then run OpenHands eval locally on my computer. Figuring out how to host the model remotely on the server and expose it publicly to my computer was a bit too difficult for me, so I looked into alternative models with an API that I could run the benchmark on instead. This led to my choice in evaluating Gemini-2.5-Flash. The Gemini API allows you to set "low reasoning effort" and "high reasoning effort", which was perfect for my experiment! 

Now that I had the model setup, I just needed to run the models on SWE-Bench. This is where I hit my second major roadblock. I needed to setup OpenHands locally on my computer, however my docker builds kept crashing on my Mac... I suspected it was due to the docker builds were built on x86_64 and my Mac uses an M2 chip. Luckily, I have a Windows PC with WSL setup at my parents' house, so I planned to use it over Thanksgiving. Due to limitations on time, RAM, and storage, the evaluations I ran on Gemini are very limited.

I ran the evaluations with the following commands on OpenHands:
```bash
./evaluation/benchmarks/swe_bench/scripts/run_infer.sh llm.eval_gemini_high HEAD CodeActAgent 10 10 1 princeton-nlp/SWE-bench_Verified test

./evaluation/benchmarks/swe_bench/scripts/run_infer.sh llm.eval_gemini_low HEAD CodeActAgent 10 10 1 princeton-nlp/SWE-bench_Verified test
```
Where `eval_limit=10` `max_iter=10` `num_workers=1`.